El objetivo de este notebook es plotear y analizar tiempos / spikes de experimentos. Por un lado cargamos un archivo donde se indica el tiempo de activación del láser, y por otro tenemos los spikes. Tenemos que plotear primero, de manera coordinada, la información de ambos archivos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt

In [ ]:
hoy = dt.date.today()

In [ ]:
name_file_spikes = 'D19_POSTsiembra_A1_(I)(001)_spike_list.csv'
name_file_laser = '202402020_MEAs_A1_(I).xls'
time_absolute_start = dt.datetime.combine(hoy, dt.time(10, 25, 6, 151000))  # A1_(I)_(001)

name_file_spikes = 'D19_POSTsiembra_A1_(IV)(000)_spike_list.csv'
name_file_laser = '202402020_MEAs_A1_(IV).xls'
time_absolute_start = dt.datetime.combine(hoy, dt.time(13, 34, 52, 893000))  # A1_(IV)_(000)


time_absolute_start

In [ ]:
df_spikes = pd.read_csv(f"../spike_lists/{name_file_spikes}", sep='\t', encoding='unicode_escape')
df_spikes['electrode_str'] = [i.split('_')[1] for i in df_spikes['Electrode']]
df_spikes['electrode_coord'] = [(int(electrode[0]) - 1, 3 - (int(electrode[1]) - 1)) for electrode in df_spikes['electrode_str']]
df_spikes

In [ ]:
df_laser = pd.read_excel(f"../tiempos_structura/{name_file_laser}", index_col=0)
df_laser['start_times'] = [(dt.datetime.combine(hoy, df_laser['Turn ON Laser Time'].iloc[i]) - time_absolute_start).total_seconds() for i in range(len(df_laser))]
df_laser['end_times'] = [(dt.datetime.combine(hoy, df_laser['Turn OFF Laser Time'].iloc[i]) - time_absolute_start).total_seconds() for i in range(len(df_laser))]
df_laser['duration'] = df_laser['ON duration (ms)'] / 1000
df_laser


In [ ]:
voltages = sorted(set(df_laser['Laser Power (mW)'].values))
electrodes = sorted(set(df_spikes['electrode_str'].values))

window = 2

fig, axs = plt.subplots(len(voltages), 1, figsize=(25, 18))


for idx, voltage in enumerate(voltages):
    df_laser_sub = df_laser[df_laser['Laser Power (mW)'] == voltage]
    t_0, t_f = min(df_laser_sub['start_times']), max(df_laser_sub['end_times'])


    for rep, start, end in zip(df_laser_sub['Repetition'].values, df_laser_sub['start_times'].values, df_laser_sub['end_times'].values):
        axs[idx].add_patch(mpl.patches.Rectangle((start, -1), end - start, len(electrodes)+1, alpha=0.25))

    for electrode_idx, electrode in enumerate(electrodes):
        df_spikes_sub = df_spikes[(df_spikes['Time (s)'] > t_0 - window) & (df_spikes['Time (s)'] < t_f + window) & (df_spikes['electrode_str'] == electrode)]
        axs[idx].scatter(x=df_spikes_sub['Time (s)'].values, y=[electrode_idx] * len(df_spikes_sub), marker='|', c='#455454')
    


    axs[idx].set_yticks(np.array(range(len(electrodes))))
    axs[idx].set_yticklabels(electrodes)
    axs[idx].set_ylim([-1, len(electrodes)])